In [1]:
import pandas as pd

In [2]:
%%time
df=pd.read_csv("Big_data.csv")

<timed exec>:1: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,34,36,38,39) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 3min 9s


In [3]:
df.head()


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,1363745270,GGY6450,99,PAS,07/09/2015,46,SDN,HONDA,P,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1363745293,KXD355,SC,PAS,07/09/2015,21,SUBN,CHEVR,P,55730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1363745438,JCK7576,PA,PAS,07/09/2015,21,SDN,ME/BE,P,42730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1363745475,GYK7658,NY,OMS,07/09/2015,21,SUBN,NISSA,P,58130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1363745487,GMT8141,NY,PAS,07/09/2015,21,P-U,LINCO,P,58130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(10626899, 51)

In [5]:
!pip install modin
!pip install dask
!pip install --upgrade ray

In [6]:
import modin.pandas as mpd
import dask.dataframe as dd
#import ray
#import ray.dataframe as rr
#ray.init()

In [7]:
%%time
df1=dd.read_csv("Big_data.csv")

Wall time: 17.3 ms


In [18]:
import yaml
import re

In [20]:
separator = '|'

In [21]:
df.columns = [re.sub(r'\W+', '_', col.strip()) for col in df.columns]

In [30]:
print(df.columns)

Index(['Summons_Number', 'Plate_ID', 'Registration_State', 'Plate_Type',
       'Issue_Date', 'Violation_Code', 'Vehicle_Body_Type', 'Vehicle_Make',
       'Issuing_Agency', 'Street_Code1', 'Street_Code2', 'Street_Code3',
       'Vehicle_Expiration_Date', 'Violation_Location', 'Violation_Precinct',
       'Issuer_Precinct', 'Issuer_Code', 'Issuer_Command', 'Issuer_Squad',
       'Violation_Time', 'Time_First_Observed', 'Violation_County',
       'Violation_In_Front_Of_Or_Opposite', 'House_Number', 'Street_Name',
       'Intersecting_Street', 'Date_First_Observed', 'Law_Section',
       'Sub_Division', 'Violation_Legal_Code', 'Days_Parking_In_Effect',
       'From_Hours_In_Effect', 'To_Hours_In_Effect', 'Vehicle_Color',
       'Unregistered_Vehicle_', 'Vehicle_Year', 'Meter_Number',
       'Feet_From_Curb', 'Violation_Post_Code', 'Violation_Description',
       'No_Standing_or_Stopping_Violation', 'Hydrant_Violation',
       'Double_Parking_Violation', 'Latitude', 'Longitude', 'Communit

In [32]:
output_yaml_file = 'schema.yaml'
separator = '|'

columns = list(df.columns)


with open(output_yaml_file, 'w') as f:
    yaml.dump({'columns': columns}, f)


with open(output_yaml_file, 'r') as f:
    print(f.read())

columns:
- Summons_Number
- Plate_ID
- Registration_State
- Plate_Type
- Issue_Date
- Violation_Code
- Vehicle_Body_Type
- Vehicle_Make
- Issuing_Agency
- Street_Code1
- Street_Code2
- Street_Code3
- Vehicle_Expiration_Date
- Violation_Location
- Violation_Precinct
- Issuer_Precinct
- Issuer_Code
- Issuer_Command
- Issuer_Squad
- Violation_Time
- Time_First_Observed
- Violation_County
- Violation_In_Front_Of_Or_Opposite
- House_Number
- Street_Name
- Intersecting_Street
- Date_First_Observed
- Law_Section
- Sub_Division
- Violation_Legal_Code
- Days_Parking_In_Effect
- From_Hours_In_Effect
- To_Hours_In_Effect
- Vehicle_Color
- Unregistered_Vehicle_
- Vehicle_Year
- Meter_Number
- Feet_From_Curb
- Violation_Post_Code
- Violation_Description
- No_Standing_or_Stopping_Violation
- Hydrant_Violation
- Double_Parking_Violation
- Latitude
- Longitude
- Community_Board
- Community_Council
- Census_Tract
- BIN
- BBL
- NTA



In [33]:
input_yaml_file = 'schema.yaml'


with open(input_yaml_file, 'r') as stream:
    try:
        yaml_data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

expected_columns = yaml_data['columns']

if len(df.columns) != len(expected_columns):
    print('Error: Number of columns does not match the YAML file')
else:
    for col in df.columns:
        if col not in expected_columns:
            print(f'Error: Column {col} is not in the YAML file')
            break
    else:
        print('Column validation successful')



Column validation successful


In [34]:
import gzip
import os

output_file = 'output_file.txt.gz'

with gzip.open(output_file, 'wt', encoding='utf-8') as f:
    df.to_csv(f, sep=separator, index=False)


file_size = os.path.getsize(output_file)

print(f'Total number of rows: {len(df)}')
print(f'Total number of columns: {len(df.columns)}')
print(f'File size: {file_size} bytes')

Total number of rows: 10626899
Total number of columns: 51
File size: 402569203 bytes
